### **Notebook Setup**

**Import Packages**

In [57]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression

from hyperopt import hp
from hyperopt import Trials

from xgboost import *

from shaphypetune import BoostSearch, BoostRFE, BoostRFA, BoostBoruta
from itertools import combinations

from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

import pickle

**Read in the csv data from file**

In [5]:
# Read the atmospheric river teleconnection pattern dataframe
ar_tcp = pd.read_csv("WestCoastAR-TCP_clustered.csv")
ar_tcp['month'] = pd.to_datetime(ar_tcp['time']).dt.month # Add month column


### **Functions**

In [67]:
def optimize_XGBmodel(x,y):

    # Seperate into Test-Train splits
    X_train, X_test, y_train, y_test = train_test_split(x,y,train_size=0.8,random_state=42, shuffle=True)

    # Initiate the XGboosted regressor
    xgb_reg = XGBRegressor()

    # Specify the hyperparameter distribution to search
    param_dist = {
        'n_estimators': stats.randint(50,500),
        'learning_rate': stats.uniform(0.01, 0.9),
        'num_leaves': stats.randint(20,40)
    }

    # Tune hyperparameters and conduct Boruta feature selection simultaneously
    xgb_regBoruta = BoostBoruta(xgb_reg, max_iter=20, perc=100,
                                importance_type='shap_importances',
                                train_importance=False,
                                param_grid=param_dist,
                                n_iter=20,
                                n_jobs=None,
                                sampling_seed=42)

    # Search for optimal combination of features and parameters
    xgb_regBoruta.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=6, verbose=False) # 2 minutes


    #
    ## Investigate results of tuning and feature selection
    #

    print('Attributes before RFE: {}\n'.format(X_train0.columns.values))

    print('Attributes after RFE: {}\n'.format(X_train0.columns[xgb_regBoruta.get_support()].values))

    print('Attributes eliminated during RFE: {}\n'.format(X_train0.columns[(xgb_regBoruta.get_support()==False)].values))


    # Remove the Phase, Theta_SSD, and origin_lon
    retained_features = x.columns[xgb_regBoruta.get_support()]
    x_trim = x[retained_features] # Using physical characteristics contextualized within a synoptic environment using teleconnection pattern indices

    
    # Seperate into Test-Train splits
    X_train, X_test, y_train, y_test = train_test_split(x_trim,y,train_size=0.8,random_state=42, shuffle=True)


    # Initiate the XGboosted Regressor with the best parameters from RFE
    xgb_reg0 = XGBRegressor(**xgb_regBoruta.best_params_)

    # Fit the model with the training data
    optimized_model = xgb_reg0.fit(X_train,y_train)



    
    return optimized_model, retained_features

### **Simultaneous Feature Selection and Hyperparameter Optimization for each Cluster**

In [68]:
# Get the clusterIDs from the AR Teleconnection Pattern DataFrame
clusterIDs = np.unique(ar_tcp['4_Clusters'])


# Optimize an XGBoosted Regression Model for each cluster
for clusterID in clusterIDs:

    # Select the Cluster from the AR Teleconnection Pattern DataFrame
    c = ar_tcp.loc[ar_tcp['4_Clusters']==clusterID]

    # Select the Predictand & its attributes
    y = c['strength']
    x = c[['BulkAxisDi','SSD_Theta','centroid_y', 'centroid_x', 'PMM', 'QBO', 'MEI', 'Phase','Curvature', 'Displacement','origin_lat', 'origin_lon',
       'Amplitude', 'pna_index_cdas', 'nao_index_cdas','month']] 
    
    # Optimize the XGBmodel on the for the Cluster
    optimized_model, retained_features = optimize_XGBmodel(x,y)

   # Save the optimized model to file
    optimized_model.save_model('Cluster-{}_XGBmodel.txt'.format(clusterID))

   # Save the optimized model's retained features to file
    with open('Cluster-{}_RetainedFeatures.pkl'.format(clusterID),'wb') as file:

       pickle.dump(retained_features,file)





20 trials detected for ('n_estimators', 'learning_rate', 'num_leaves')

<class 'int'>
<class 'dict'>


`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_sto

trial: 0001 ### iterations: 00024 ### eval_score: 43.91151
<class 'int'>
<class 'dict'>


`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_sto

trial: 0002 ### iterations: 00018 ### eval_score: 45.92035
<class 'int'>
<class 'dict'>


...

trial: 0018 ### iterations: 00320 ### eval_score: 50.97292
<class 'int'>
<class 'dict'>


`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_sto

trial: 0019 ### iterations: 00063 ### eval_score: 56.55567
<class 'int'>
<class 'dict'>


`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
`early_sto

trial: 0020 ### iterations: 00079 ### eval_score: 53.96341
Attributes before RFE: ['BulkAxisDi' 'SSD_Theta' 'centroid_y' 'centroid_x' 'PMM' 'QBO' 'MEI'
 'Phase' 'Curvature' 'Displacement' 'origin_lat' 'origin_lon' 'Amplitude'
 'pna_index_cdas' 'nao_index_cdas' 'month']

Attributes after RFE: ['BulkAxisDi' 'SSD_Theta' 'centroid_y' 'centroid_x' 'PMM' 'QBO' 'MEI'
 'Phase' 'Curvature' 'Displacement' 'origin_lat' 'origin_lon' 'Amplitude'
 'pna_index_cdas' 'nao_index_cdas' 'month']

Attributes eliminated during RFE: []

[13:40:17] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "num_leaves" } are not used.

